## Enviorment
The following section are used to specify the hyperparameter and the path the the software that were used. 
No need to changed if the pipeline stay the same and were applied to different data


In [16]:
##!/bin/sh
# MAKE SURE FUSION.compute_weights.R IS IN YOUR PATH
# FILL IN THESE PATHS
# For mac user, the mac version of GCTA shall be downloaded saperately, the one came with the Fusion package will not work.
GCTA="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/gcta_1.93.2beta_mac/gcta64"
PLINK="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/plink_mac_20200616/plink"



GEMMA="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/GEMMA"
Rscp="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/FUSION.compute_weights.R"
# Path to the input data
Datarep="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/Testing/Data/"
# ALTERNATIVELY: ENSURE THAT plink, gcta, gemma CAN BE CALLED FROM PATH AND REMOVE --PATH_* FLAGS BELOW

# PATH TO WORKING DIRECTORY
wd="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/Testing"

# PATH TO OUTPUT DIRECTORY (population-specific subdirs will be made)
OUT_DIR="./WEIGHTS"

# ROWS IN THE MATRIX TO ANALYZE (FOR BATCHED RUNS)
BATCH_START=1
BATCH_END=10

## Input 

This is where the actual RNA Seq data of our own should be used as input.

Note: GD462.GeneQuantRPKM.50FN.samplename.resk10.txt is the rna seq data for gene expression from the "465 lymphoblastoid cell lines from the 1000 Genomes" projects. There is 23722 genes from 462 samples. The first 4 col is "TargetID, Gene_Symbol, Chr, Coord". The rest are each samples.

GD462.GeneQuantRPKM.50FN.samplename.resk10.txt is the RNA seq data for the first 10 ppl

LDREF/1000G.EUR.*.bim contains the linkage disequilibrium info for each snp.

In [17]:
# PATH TO DIRECTORY CONTAINING LDREF DATA (FROM FUSION WEBSITE or https://data.broadinstitute.org/alkesgroup/FUSION/LDREF.tar.bz2)
LDREF="/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF"
# THIS IS USED TO RESTRICT INPUT SNPS TO REFERENCE IDS ONLY

# PATH TO GEUVADIS GENE EXPRESSION MATRIX:
PRE_GEXP="GD462.GeneQuantRPKM.50FN.samplename.resk10.txt"
# GEUVADIS DATA WAS DOWNLOADED FROM https://www.ebi.ac.uk/arrayexpress/experiments/E-GEUV-1/files/analysis_results/

# PATH TO PREFIX FOR GEUVADIS GENOTYPES SPLIT BY CHROMOSOME
# SUBSAMPLE THESE TO THE LDREF SNPS FOR EFFICIENCY
PRE_GENO="1000G.EUR"

## Pipeline


In [18]:
# --- BEGIN SCRIPT:
cd $wd
NR="${BATCH_START}_${BATCH_END}"
mkdir -p tmp/$NR
mkdir -p hsq/$NR
mkdir -p out/$NR
# THIS IS DIRECTORY WHERE THE OUTPUT WILL GO:
mkdir $OUT_DIR

mkdir: ./WEIGHTS: File exists



In [19]:
# makes necessary component that are useful for follow up pipeline
cd $wd
head $Datarep$PRE_GEXP | awk '{$1=$2=$3=$4=""; print substr($0,5)}' | fmt -1 > PRE_GEXPID

Note 1: the first two column of the out.pheno shall be the same as that of the .fam file

In [20]:
cd $wd
# Loop through each gene expression phenotype in the batch (Only looking at the first 10 genes)
cat $Datarep$PRE_GEXP | awk -v s=$BATCH_START -v e=$BATCH_END 'NR > s && NR <= e' |  while read PARAM; do
# Get the gene positions +/- 500kb
CHR=`echo $PARAM | awk '{ print $3 }'`
P0=`echo $PARAM | awk '{ print $4 - 0.5e6 }'`
P1=`echo $PARAM | awk '{ print $4 + 0.5e6 }'`
GNAME=`echo $PARAM | awk '{ print $1 }'`

OUT="tmp/$NR/$PRE_GEXP.$GNAME"

echo $GNAME $CHR $P0 $P1

## Pull out the current gene expression phenotype (Note 1 )
echo $PARAM | tr ' ' '\n' | tail -n+5 | paste PRE_GEXPID PRE_GEXPID - > $OUT.pheno
## Get the locus genotypes for all samples and set current gene expression as the phenotype
$PLINK --bfile $LDREF/$PRE_GENO.$CHR --pheno $OUT.pheno --make-bed --out $OUT --chr $CHR --from-bp $P0 --to-bp $P1 --extract $LDREF/1000G.EUR.$CHR.bim --keep $OUT.pheno --allow-no-sex
## Process all samples together (for reference purposes only since this is mult-ethnic data)
mkdir $OUT_DIR/ALL
FINAL_OUT="$OUT_DIR/ALL/ALL.$GNAME"
#
Rscript $Rscp --bfile $OUT --tmp $OUT.$pop.tmp --out $FINAL_OUT --verbose 0 --save_hsq --PATH_gcta $GCTA --PATH_gemma $GEMMA --models blup,lasso,top1,enet
### ALTERNATIVELY ADD COVARIATES HERE USING THE --covar FLAG
## MINIMAL COMMAND IS: `Rscript FUSION.compute_weights.R --bfile $OUT --tmp $OUT.$pop.tmp --out $FINAL_OUT`
#
## Append heritability output to hsq file
cat $FINAL_OUT.hsq >> hsq/$NR.hsq
#
## Clean-up just in case
rm -f $FINAL_OUT.hsq $OUT.tmp.*
#
# Remove all intermediate files
echo "end of circle"

# GO TO THE NEXT GENE
done




ENSG00000152931.6 5 59283540 60283540
PLINK v1.90b6.18 64-bit (16 Jun 2020)          www.cog-genomics.org/plink/1.9/
(C) 2005-2020 Shaun Purcell, Christopher Chang   GNU General Public License v3
Logging to tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000152931.6.log.
Options in effect:
  --allow-no-sex
  --bfile /Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF/1000G.EUR.5
  --chr 5
  --extract /Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF/1000G.EUR.5.bim
  --from-bp 59283540
  --keep tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000152931.6.pheno
  --make-bed
  --out tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000152931.6
  --pheno tmp/1_10/GD462.GeneQuantRPKM.50FN.samplename.resk10.txt.ENSG00000152931.6.pheno
  --to-bp 60283540

16384 MB RAM detected; reserving 8192 MB for main workspace.
316 out of 75164 variants loaded from .bim file.
489 people (0 mal

bash: /Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/FUSION.compute_weights.R: Permission denied



In [22]:
cd $wd/hsq
cat 1_10.hsq | grep ALL -c

6



In [11]:
cd hsq

In [74]:
cat 1_10.hsq | grep . -c

147



In [120]:
cd /Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF/

In [123]:
head /Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF/1000G.10.bed

head: /Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF/1000G.10.bed: No such file or directory



In [55]:
echo $LDREF.ID

/Users/haosun/Documents/WG_Reasearch_Assisstant/Fusion/install/fusion_twas-master/LDREF.ID



patential working genes
ENSG00000225267.1
ENSG00000259382.1
ENSG00000135046.9